In [1]:
text_file="/dataset/dir/data/file.csv"

In [2]:
plays = sc.textFile(text_file)

In [3]:
plays.take(5)

['00000c289a1829a808ac09c00daf10bc3c4e223b\t3bd73256-3905-4f3a-97e2-8b341527f805\tbetty blowtorch\t2137',
 '00000c289a1829a808ac09c00daf10bc3c4e223b\tf2fb0ff0-5679-42ec-a55c-15109ce6e320\tdie Ärzte\t1099',
 '00000c289a1829a808ac09c00daf10bc3c4e223b\tb3ae82c2-e60b-4551-a76d-6620f1b456aa\tmelissa etheridge\t897',
 '00000c289a1829a808ac09c00daf10bc3c4e223b\t3d6bbeb7-f90e-4d10-b440-e153c0d10b53\telvenking\t717',
 '00000c289a1829a808ac09c00daf10bc3c4e223b\tbbd2ffd7-17f4-4506-8572-c1ea58c3f9a8\tjuliette & the licks\t706']

In [4]:
#  user-mboxsha1 \t musicbrainz-artist-id \t artist-name \t plays
# SPlit on /t
ratings_data = plays.map(lambda line: line.split("\t"))


In [5]:
ratings_data.take(5)

[['00000c289a1829a808ac09c00daf10bc3c4e223b',
  '3bd73256-3905-4f3a-97e2-8b341527f805',
  'betty blowtorch',
  '2137'],
 ['00000c289a1829a808ac09c00daf10bc3c4e223b',
  'f2fb0ff0-5679-42ec-a55c-15109ce6e320',
  'die Ärzte',
  '1099'],
 ['00000c289a1829a808ac09c00daf10bc3c4e223b',
  'b3ae82c2-e60b-4551-a76d-6620f1b456aa',
  'melissa etheridge',
  '897'],
 ['00000c289a1829a808ac09c00daf10bc3c4e223b',
  '3d6bbeb7-f90e-4d10-b440-e153c0d10b53',
  'elvenking',
  '717'],
 ['00000c289a1829a808ac09c00daf10bc3c4e223b',
  'bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8',
  'juliette & the licks',
  '706']]

In [6]:
#we only want userid, artist_id  and number of plays; these are found in the following columns
ratings_data_pertinent = ratings_data.map(lambda x: (x[0], x[2],(x[3])))

In [9]:
###BEGIN DATA Munging -- 

In [7]:
# Let's make a numerical id out of the encrypted user_id feature:
#Note: it may be easier to just assign a new id, rather than using the encrypted user_id field.
#However, this project is to demonstrate data cleaning as well as model development.
#Furthermore, what if assigning a new id is  not desirable in a situation?
#For better or worse, perhaps we will be creating a user_id we can use, the "hard way,"
#The approache we take for the artist_id field could certainly be used and would be easier. 
#But, easier does not always equal better.

In [8]:
# Perhaps regex would be more efficient and easier, but I find dictionaries extremely useful when I need to replace words or
# or remove words. It seems regex can  be more "large-handed" and clumsy. Dictionaries are very precise.
#Im defining a  method here to replace the letters in the artist MD5-SHA with numbers:
import string

def numerizer(word):
    translation = str.maketrans("","", string.punctuation)
    new_word = word.translate(translation)
    new_word_lower = new_word.lower()
    return new_word_lower.strip(" ")

def md5_replace(word,_dict):
    i = list(_dict.keys())
    j = list(_dict.values())
    k = len(j)-1
    _word = numerizer(word)
    while k >= 0:
        _word1 = str.replace(_word, i[k], j[k])
        _word =_word1  
        k-= 1
    return _word1


In [9]:
alpha_dict = {"a":"1", "b":"2","c":"3","d":"4", "e":"5","f":"6","g":"7",
              "h":"8","i":"9","j":"10","k":"11","l":"12","m":"13","n":"14","o":"15",
                  "p":"16","q":"17","r":"18", "s":"19","t":"20","u":"21","v":"22","w":"23","x":"24",
                  "y":"25","z":"26"," ":"0", "ä":"27","ö":"28"}

In [10]:
# Get rid of leading zeroes which will be problematic and apply alpha character function
numeric_uid = ratings_data_pertinent.map(lambda x: md5_replace(x[0], alpha_dict)).map(lambda line: line.lstrip("0"))

In [11]:
#We can see below that we have a numeric uid now. This user has the first 49 entries. However, we need to cut down 
# The size of this number; alot!
numeric_uid.take(5)

['32891182918081309300416102333452232',
 '32891182918081309300416102333452232',
 '32891182918081309300416102333452232',
 '32891182918081309300416102333452232',
 '32891182918081309300416102333452232']

In [12]:
def shrinker(num):
    places = len(num)-9
 
    if places > 0:
        zeroes = places 
      
        cutter = "1"
        for x in range(zeroes):
             cutter+=str(0)
        return str(round(int(num)/float(cutter)))

    else:
        return num
    

In [13]:
numeric_uid = numeric_uid.map(lambda x : shrinker(x))

In [14]:
numeric_uid.take(10)

['328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829',
 '328911829']

In [15]:
numeric_uid_ = numeric_uid.zipWithIndex()

In [16]:
numeric_uid_.take(5)

[('328911829', 0),
 ('328911829', 1),
 ('328911829', 2),
 ('328911829', 3),
 ('328911829', 4)]

In [17]:
numeric_uid_ = numeric_uid_.map(lambda x : (x[1],x[0]))

In [18]:
numeric_uid_.take(5)

[(0, '328911829'),
 (1, '328911829'),
 (2, '328911829'),
 (3, '328911829'),
 (4, '328911829')]

In [19]:
#To get all of the artists we will use groupby. Then we can assign a number to each of the unique artists
artist_name = ratings_data_pertinent.map(lambda x: (x[1])).distinct()

In [20]:
artist_name.top(5)

['�������', '��疲暎�', '��|', '￼beastie boys', '￼antonio mairena']

In [21]:
#Make sure we still have all the artists
artist_name.count()

292589

In [22]:
#Now we can add a unique id for each artist name
artist_index=sc.parallelize(range(1,292590))

In [23]:
artist_index = artist_index.zipWithIndex()
artist_name = artist_name.zipWithIndex()

In [24]:
artist_index = artist_index.map(lambda x: (x[1],x[0]))
artist_name = artist_name.map(lambda x: (x[1],x[0]))

In [25]:
artist_name.take(5)

[(0, 'camo & krooked'),
 (1, 'ney-the sufi cry out'),
 (2, 'camela leierth'),
 (3,
  'chamy. ishi, rei kondoh, taiki endo, yasutaka hatade, satoshi miyashita'),
 (4, 'january star')]

In [26]:
#numeric_aid_indexed = numeric_aid_index.zip(numeric_aid_)
aid_indexed = artist_index.join(artist_name)

In [27]:
aid_indexed.top(5)

[(292588, (292589, '吴景略')),
 (292587, (292588, 'david t. chastain')),
 (292586, (292587, 'walter meego')),
 (292585, (292586, 'berliner saxophon-ensemble')),
 (292584, (292585, 'badboe'))]

In [28]:
aid_indexed = aid_indexed.map(lambda x: (x[1][1],x[0]))

In [29]:
aid_indexed.top(5)

[('�������', 194632),
 ('��疲暎�', 287148),
 ('��|', 65655),
 ('￼beastie boys', 289787),
 ('￼antonio mairena', 222260)]

In [30]:
artist_titles = ratings_data_pertinent.map(lambda x:(x[1]))

In [31]:
artist_titles.take(5)

['betty blowtorch',
 'die Ärzte',
 'melissa etheridge',
 'elvenking',
 'juliette & the licks']

In [32]:
#To use the index as a sort of lookup table, we need to estabilish a common index
artists_ = artist_titles.keyBy(lambda r : r)
index_ = aid_indexed.keyBy(lambda r: r[0])

In [33]:
artists_.take(5)

[('betty blowtorch', 'betty blowtorch'),
 ('die Ärzte', 'die Ärzte'),
 ('melissa etheridge', 'melissa etheridge'),
 ('elvenking', 'elvenking'),
 ('juliette & the licks', 'juliette & the licks')]

In [34]:
index_.take(5)

[('camo & krooked', ('camo & krooked', 0)),
 ('nat king cole & dean martin', ('nat king cole & dean martin', 204800)),
 ('bresser', ('bresser', 106500)),
 ('jardín de heno', ('jardín de heno', 213000)),
 ('armand van helden & sneaker pimps',
  ('armand van helden & sneaker pimps', 240300))]

In [36]:
artists_index = artists_.join(index_)

In [37]:
#Heres our artists index
artists_index.take(5)

[('camo & krooked', ('camo & krooked', ('camo & krooked', 0))),
 ('camo & krooked', ('camo & krooked', ('camo & krooked', 0))),
 ('camo & krooked', ('camo & krooked', ('camo & krooked', 0))),
 ('camo & krooked', ('camo & krooked', ('camo & krooked', 0))),
 ('camo & krooked', ('camo & krooked', ('camo & krooked', 0)))]

In [38]:
# Take just the tuple we need
artist_index = artists_index.map(lambda x: (x[1][1]))

In [39]:
artist_index.take(5)

[('camo & krooked', 0),
 ('camo & krooked', 0),
 ('camo & krooked', 0),
 ('camo & krooked', 0),
 ('camo & krooked', 0)]

In [40]:
ratings_data = ratings_data_pertinent.map(lambda x: (x[2]))

In [41]:
ratings_data.take(5)

['2137', '1099', '897', '717', '706']

In [42]:
artist_names = artist_index.map(lambda x: (x[0]))
artist_id = artist_index.map(lambda x: (x[1]))

In [43]:
ratings = ratings_data.zipWithIndex()
artist_index_ = artist_index.zipWithIndex()
uids = numeric_uid.zipWithIndex()

In [44]:
uids.take(5)

[('328911829', 0),
 ('328911829', 1),
 ('328911829', 2),
 ('328911829', 3),
 ('328911829', 4)]

In [45]:
artist_index_.take(5)

[(('camo & krooked', 0), 0),
 (('camo & krooked', 0), 1),
 (('camo & krooked', 0), 2),
 (('camo & krooked', 0), 3),
 (('camo & krooked', 0), 4)]

In [46]:
artist_index_ = artist_index_.map(lambda x: (x[1],x[0]))
uids = uids.map(lambda x: (x[1],x[0]))
ratings = ratings.map(lambda x: (x[1],x[0]))

In [47]:
ratings.take(5)

[(0, '2137'), (1, '1099'), (2, '897'), (3, '717'), (4, '706')]

In [49]:
RDD = uids.join(artist_index_).join(ratings)

In [50]:
RDD.take(5)

[(0, (('328911829', ('camo & krooked', 0)), '2137')),
 (13107200, (('266493556', ('grupo montez de durango', 219632)), '517')),
 (8912900, (('821342251', ('bob marley', 154398)), '424')),
 (4718600, (('444164354', ("the cat's miaow", 82546)), '81')),
 (524300, (('717350741', ('ozzy osbourne', 9055)), '74'))]

In [51]:
def split_tuple(data):
    v = data[0]
    w = data[1][0][0]
    x = data[1][0][1][1]
    y = data[1][0][1][0]
    z = data[1][1]
    return(v, w, x , y, z)

In [52]:
_RDD = RDD.map(lambda x: (split_tuple(x)))

In [53]:
#Heres our new dataset! It is in the form of index|user_id|artist_name|artist_id|number_of_listens (our implicit rating)
_RDD.take(5)

[(0, '328911829', 0, 'camo & krooked', '2137'),
 (13107200, '266493556', 219632, 'grupo montez de durango', '517'),
 (8912900, '821342251', 154398, 'bob marley', '424'),
 (4718600, '444164354', 82546, "the cat's miaow", '81'),
 (524300, '717350741', 9055, 'ozzy osbourne', '74')]

In [84]:
rdd = _RDD.map(lambda x:(str(x)))

In [168]:
# python 3
import csv, io

def list_to_csv_str(x):
    """Given a list of strings, returns a properly-csv-formatted string."""
    output = io.StringIO("")
    csv.writer(output).writerow(x)
    return output.getvalue().strip() # remove extra newline


In [170]:
rdd = _RDD.map(list_to_csv_str)
rdd.saveAsTextFile("hdfs://server:9000/finalRDD.csv")